In [9]:
;cd "G:\\My Drive\\_MOPJ\\Julia\\FEM"

G:\My Drive\_MOPJ\Julia\FEM


In [4]:
]st

Status `D:\Users\Administrator\.julia\environments\v1.5\Project.toml`
  [7861f83d] AD4SM v0.0.1
  [c52e3926] Atom v0.12.19
  [6e4b80f9] BenchmarkTools v0.5.0
  [336ed68f] CSV v0.7.7
  [8f4d0f93] Conda v1.4.1
  [3a865a2d] CuArrays v2.2.2
  [a93c6f00] DataFrames v0.21.6
  [0c46a032] DifferentialEquations v6.15.0
  [31c24e10] Distributions v0.23.8
  [7a1cc6ca] FFTW v1.2.2
  [28b8d3ca] GR v0.51.0
  [09f84164] HypothesisTests v0.10.0
  [7073ff75] IJulia v1.21.3
  [6218d12a] ImageMagick v1.1.5
  [916415d5] Images v0.22.4
  [c601a237] Interact v0.10.3
  [4138dd39] JLD v0.10.0
  [18b7da76] JuliaAcademyData v0.1.0 `https://github.com/JuliaComputing/JuliaAcademyData.jl#master`
  [e5e0dc1b] Juno v0.8.3
  [5ab0869b] KernelDensity v0.5.1
  [3625dbae] LibFEM v0.1.0 `G:\\My Drive\\_MOPJ\\Julia\\LibFEM`
  [23992714] MAT v0.8.0
  [f0e99cf1] MLBase v0.8.0
  [ee78f7c6] Makie v0.11.1
  [15e1cf62] NPZ v0.4.0
  [5fb14364] OhMyREPL v0.5.5
  [429524aa] Optim v0.22.0
  [91a5bcdd] Plots v1.6.0
  [438e738f] PyCa

In [ ]:
]add https://github.com/amdeld/LibFEM.jl

In [1]:
using LibFEM # ,Plots

In [1]:
# Units system mm/tonne/s/K
# ===============================================================================
# 2D FRAME
# ===============================================================================
# (2)---(3)
#      /
#     /
#    /
#   /
#  /
# /
#(1)
# ===============================================================================
# The element used here is a linear 2D straight truss with constant cross section
# The degrees of freedom are the u & v displacements 
# ===============================================================================
#PARAMETERS
L=10000. #length in mm
A=100. #cross-sectional area in mm^2
E=210000. #modulus of elasticity in MPa [steel]
FM=10000. #force modulus in N
# ===============================================PRE-PROCESSING==================
# DEFINING AND DISCRETIZING[MESHING] THE STRUCTURE
# connectivity table
# elt||node_i||node_j
# 1|1|3
# 2|2|3
# grid()
X1pos=0.;Y1pos=0.
X2pos=0.;Y2pos=L
X3pos=L;Y3pos=L
#lengths
L1=D2_TrussElementLength(X1pos,Y1pos,X3pos,Y3pos); #length of element 1
L2=D2_TrussElementLength(X2pos,Y2pos,X3pos,Y3pos); #length of element 2
#APPLYING GEOMETRIC&MATERIAL PROPERTIES
A1=sqrt(2)*A; #cross-sectional area of element 1
A2=A; #cross-sectional area of element 2
E1=E; #material of element 1
E2=E; #material of element 2
#writing-defining the element stiffness matrices
K1=D2_TrussElementStiffness(E1,A1,L1,45);
println("\nK1=\r")
display(K1)
K2=D2_TrussElementStiffness(E2,A2,L2,0);
println("\nK2=\r")
display(K2)
#ASSEMBLING THE GLOBAL STIFFNESS MATRIX
#matrices initialization
K=zeros(6,6);K1P=zeros(6,6);
#positionning stiffness matrices
K1P=D2_TrussAssemble(K,K1,1,3)
println("\nK1P=\r")
display(K1P)
K=zeros(6,6);K2P=zeros(6,6);
K2P=D2_TrussAssemble(K,K2,2,3)
println("\nK2P=\r")
display(K2P)
#assembling
K=K1P+K2P
println("\nK=\r")
display(K)
#SOLVING DISPACEMENT EQUATIONS
#extracting displacement submatrix via index vector
K_s=K[5:6,5:6]
#Setting-up the force subvector by applying Load & Boundary Conditions[LBC]]
F_s=[0 FM]'
#solving by gaussian elimination
U_s=K_s\F_s
#SOLVING FORCE EQUATIONS
#setting-up the global nodal displacement vector
U=[0 0 0 0 U_s[1:2]']'
println("\nU=\r")
display(U)
#computing the global nodal force vector
F=K*U
println("\nF=\r")
display(F)
#COMPUTING STRESSES
#writing the element nodal displacement vectors
U1=[U[1] U[2] U[5] U[6]]'
U2=[U[3] U[4] U[5] U[6]]'
#computing element strains
ϵ1=D2_TrussElementStrain(L1,45,U1)
println("\nϵ1=\r")
display(ϵ1)
ϵ2=D2_TrussElementStrain(L2,0,U2)
println("\nϵ2=\r")
display(ϵ2)#computing element forces
f1=D2_TrussElementForce(E1,A1,L1,45,U1)
f2=D2_TrussElementForce(E2,A2,L2,0,U2)
#computing element stresses
σ1=D2_TrussElementStress(E1,L1,45,U1)
println("\nσ1=\r")
display(σ1)
σ2=D2_TrussElementStress(E2,L2,0,U2)
println("\nσ2=\r")
display(σ2)

4×4 Array{Float64,2}:
  1050.0   1050.0  -1050.0  -1050.0
  1050.0   1050.0  -1050.0  -1050.0
 -1050.0  -1050.0   1050.0   1050.0
 -1050.0  -1050.0   1050.0   1050.0


K1=


4×4 Array{Float64,2}:
  2100.0   0.0  -2100.0  -0.0
     0.0   0.0     -0.0  -0.0
 -2100.0  -0.0   2100.0   0.0
    -0.0  -0.0      0.0   0.0

6×6 Array{Float64,2}:
  1050.0   1050.0  0.0  0.0  -1050.0  -1050.0
  1050.0   1050.0  0.0  0.0  -1050.0  -1050.0
     0.0      0.0  0.0  0.0      0.0      0.0
     0.0      0.0  0.0  0.0      0.0      0.0
 -1050.0  -1050.0  0.0  0.0   1050.0   1050.0
 -1050.0  -1050.0  0.0  0.0   1050.0   1050.0

6×6 Array{Float64,2}:
 0.0  0.0      0.0  0.0      0.0  0.0
 0.0  0.0      0.0  0.0      0.0  0.0
 0.0  0.0   2100.0  0.0  -2100.0  0.0
 0.0  0.0      0.0  0.0      0.0  0.0
 0.0  0.0  -2100.0  0.0   2100.0  0.0
 0.0  0.0      0.0  0.0      0.0  0.0

...

1×1 Array{Float64,2}:
 0.0004761904761904763

1×1 Array{Float64,2}:
 -0.0004761904761904763

1×1 Array{Float64,2}:
 100.00000000000003

1×1 Array{Float64,2}:
 -100.00000000000001



ϵ1=

ϵ2=

σ1=

σ2=
